In [3]:
import sys
sys.path.append('/usr/lib/python2.7/dist-packages')

In [2]:
import rospy
import cv2
import json
from sensor_msgs.msg import Image
from std_msgs.msg import String
from cv_bridge import CvBridge, CvBridgeError

In [3]:
class DaHuaNetCam:
    
    def __init__(self, user, pwd, ip, channel):
        # Default parameters, can be set.
        self._fps = 30
        self._netCamId = 1
        self._img_size = (1280, 720)
        # Get the value from config.
        self.user = user
        self.pwd = pwd
        self.ip = ip
        self.channel = channel
    
    def openNetCam(self):
        import cv2
        # NetCam URL:
        url = 'rtsp://%s:%s@%s/cam/realmonitor?channel=%d&subtype=0' \
              % (self.user, self.pwd, self.ip, self.channel)
        # netCam = cv2.VideoCapture(url)
        netCam = cv2.VideoCapture(0)
        if netCam.isOpened():
            return netCam
        raise Exception("NetCam can't open! [NetCam URL]: %s" % url)
    
    def set_fps(self, fps):
        self._fps = fps
        
    def set_netCamId(self, netCamId):
        self._netCamId = netCamId
        
    def set_img_size(self, img_size):
        self._img_size = img_size

In [ ]:
class NetCam_Converter:
    
    def __init__(self, image_topic):
        self.image_pub = rospy.Publisher(image_topic, Image)
        self.bridge = CvBridge()
    
    def publish(self, image):
        try:
            image = self.bridge.cv2_to_imgmsg(image, 'bgr8')
            self.image_pub.publish(image)
        except CvBridgeError as e:
            print(e)

In [ ]:
# import tf_pose
from tf_pose import common
print(common.__file__)
print(sys.path)

In [ ]:
user, pwd, ip, channel = ['admin', 'admin2019', '172.16.1.180', 1]
netCam = DaHuaNetCam(user, pwd, ip, channel)
netCap = netCam.openNetCam()

while netCap.isOpened():
    ret, frame = netCap.read()
    if ret:
        cv2.imshow("NetCam Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

netCap.release()
cv2.destroyAllWindows()

In [4]:
import logging, time
import easydict

from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import model_wh

logger = logging.getLogger('TfPoseEstimatorRun')
logger.handlers.clear()
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

W0705 18:01:26.456783 140565966112576 deprecation_wrapper.py:119] From /home/commaai-03/Mikoy/work/open_ptrack/tf-pose-estimation/tf_pose/mobilenet/mobilenet.py:369: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



In [5]:
args = easydict.EasyDict({
    "model": "/home/commaai-03/Mikoy/work/open_ptrack/tf-pose-estimation/models/graph/mobilenet_thin/graph_opt.pb",
    "resize": "432x368",
    "resize_out_ratio": 4.0,
    "folder": "/home/commaai-03/Mikoy/work/open_ptrack/tf-pose-estimation/huapu_imgs/"
})

In [8]:
w, h = model_wh(args.resize)
if w == 0 or h == 0:
    e = TfPoseEstimator(args.model, target_size=(432, 368))
else:
    e = TfPoseEstimator(args.model, target_size=(w, h))
    
def draw_boxes(human, npimg, image_w, image_h):
    face_box = human.get_face_box(image_w, image_h)
    upper_body_box= human.get_upper_body_box(image_w, image_h)

    # face_box
    if face_box:
        x1, y1 = int(face_box['x']-face_box['w']/2), int(face_box['y']-face_box['h']/2)
        x2, y2 = int(face_box['x']+face_box['w']/2), int(face_box['y']+face_box['h']/2)
        cv2.rectangle(npimg, (x1,y1),(x2,y2),(0,255,0), 2)
    # upper_body_box
    if upper_body_box:
        x3, y3 = int(upper_body_box['x'] - upper_body_box['w']/2), int(upper_body_box['y'] - upper_body_box['h']/2)
        x4, y4 = int(upper_body_box['x'] + upper_body_box['w']/2), int(upper_body_box['y'] + upper_body_box['h']/2)
        cv2.rectangle(npimg, (x3,y3),(x4,y4),(0,0,255), 2)
    return npimg

[2019-07-05 18:01:40,754] [TfPoseEstimator] [INFO] loading graph from /home/commaai-03/Mikoy/work/open_ptrack/tf-pose-estimation/models/graph/mobilenet_thin/graph_opt.pb(default size=432x368)


In [9]:
user, pwd, ip, channel = ['admin', 'admin2019', '172.16.1.180', 1]
netCam = DaHuaNetCam(user, pwd, ip, channel)
netCap = netCam.openNetCam()
netCap.set(3, 1280)
netCap.set(4, 720)

switch = False

while netCap.isOpened():
    ret, frame = netCap.read()
    if ret:
        tf_pose_msgs = easydict.EasyDict({'header': {'stamp': {'secs': 0, 'nsecs': 0}, 'frame_id': ''}, 'poses': []})
        t = rospy.Time.now()
        tf_pose_msgs.header.stamp.secs, tf_pose_msgs.header.stamp.nsecs = t.secs, t.nsecs
        image_h, image_w = frame.shape[:2]
        humans = e.inference(frame, resize_to_default=(w > 0 and h > 0),
                             upsample_size=args.resize_out_ratio)
        
        tf_pose_msgs = get_msg(tf_pose_msgs, humans, image_w, image_h)
        tf_pose_msgs = json.dumps(tf_pose_msgs)
        pub.publish(tf_pose_msgs)
        
        #elapsed = rospy.Time.now() - t
        #logger.info('inference image in %.4f seconds.' % elapsed)
        
        if switch:
            for human in humans:
                draw_boxes(human, frame, image_w, image_h)
            cv2.imshow('ROS Face And Body Box', frame)
        else:
            frame = TfPoseEstimator.draw_humans(frame, humans, imgcopy=False)
            cv2.imshow("TF_PoseEstimator Frame", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

netCap.release()
cv2.destroyAllWindows()

#### Face Box And Upper Body Box Ros Data

In [6]:
def get_msg(tf_pose_msgs, humans, image_w, image_h, need_handle=False):
    
    for human in humans:
        tf_pose_msg = {'face_box': None, 'upper_body_box': None}
        face_box = human.get_face_box(image_w,image_h)
        upper_body_box= human.get_upper_body_box(image_w, image_h)
        if not need_handle:
            if face_box:
                tf_pose_msg['face_box'] = face_box
            if upper_body_box:
                tf_pose_msg['upper_body_box'] = upper_body_box
        else:
            # face_box
            if face_box:
                x1, y1 = int(face_box['x']-face_box['w']/2), int(face_box['y']-face_box['h']/2)
                x2, y2 = int(face_box['x']+face_box['w']/2), int(face_box['y']+face_box['h']/2)
                tf_pose_msg['face_box'] = {'x1y1': (x1,y1), 'x2y2': (x2,y2)}
            # upper_body_box
            if upper_body_box:
                x1, y1 = int(upper_body_box['x'] - upper_body_box['w']/2), int(upper_body_box['y'] - upper_body_box['h']/2)
                x2, y2 = int(upper_body_box['x'] + upper_body_box['w']/2), int(upper_body_box['y'] + upper_body_box['h']/2)
                tf_pose_msg['face_box'] = {'x1y1': (x1,y1), 'x2y2': (x2,y2)}

        tf_pose_msgs.poses.append(tf_pose_msg)
    return tf_pose_msgs

#### Ros Msg Test

In [7]:
import rospy
from std_msgs.msg import String

pub = rospy.Publisher('Test', String, queue_size=10)
rospy.init_node('TF_POSE')
r = rospy.Rate(10) # 10hz

'''
while not rospy.is_shutdown():
   pub.publish(tf_pose_msgs)
   r.sleep()
'''

'\nwhile not rospy.is_shutdown():\n   pub.publish(tf_pose_msgs)\n   r.sleep()\n'

In [1]:
import tensorflow as tf